In [1]:
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

In [3]:
def get_shape(tensor):
    "return the shape of tensor as list"
    return tensor.get_shape().as_list()

def print_var(string):
    "print var_name and var_shape in collection named string"
    print("\n".join(["{} : {}".format(v.name, get_shape(v)) for v in tf.get_collection(string)]))
    
def clip(x, vmax = 1-1e-7, vmin = 1e-7):
    "clip the x between vmax, vmin"
    return tf.clip_by_value(x, clip_value_max=vmax, clip_value_min=vmin)

In [4]:
mnist = input_data.read_data_sets('../MNIST_data', one_hot=True)

Extracting ../MNIST_data/train-images-idx3-ubyte.gz
Extracting ../MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../MNIST_data/t10k-labels-idx1-ubyte.gz


In [5]:
train = {}
test = {}
val = {}
train['input'] = mnist.train.images
train['output'] = mnist.train.labels
test['input'] = mnist.test.images
test['output'] = mnist.test.labels
val['input'] = mnist.validation.images
val['output'] = mnist.validation.labels

In [6]:
# Model parameter
image_size = 28*28
hidden = 300
nclasses = 10

# Train parameter
learning_rate = 1e-4
epoch = 5
ntrain = len(train['input'])
batch_size = 100
nbatch = ntrain//batch_size
log_every = 50

In [7]:
logs_path = "./log"

with tf.name_scope('input'):
    x = tf.placeholder(dtype = tf.float32, shape = [None, image_size], name = 'image')
    y = tf.placeholder(dtype = tf.float32, shape = [None, nclasses], name = 'label')

layer1 = {}
layer2 = {}    
    
with tf.name_scope("layer1"):
    layer1['w'] = tf.Variable(initial_value = tf.random_normal(shape = [image_size, hidden], mean = 0.0, stddev = 1.0),
                              name = 'weight')
    layer1['b'] = tf.Variable(initial_value = tf.constant(value=0.0, shape = [hidden]),
                              name = 'bias')
    layer1['output'] = tf.nn.sigmoid(tf.add(tf.matmul(x, layer1['w']), layer1['b']))

with tf.name_scope("layer2"):
    layer2['w'] = tf.Variable(initial_value = tf.random_normal(shape = [hidden, nclasses], mean = 0.0, stddev = 1.0),
                              name = 'weight')
    layer2['b'] = tf.Variable(initial_value = tf.constant(value=0.0, shape = [nclasses]),
                              name = 'bias')
    layer2['output'] = tf.nn.softmax(tf.add(tf.matmul(layer1['output'], layer2['w']), layer2['b']))
with tf.name_scope('cross_entropy'):
    cross_entropy = -tf.reduce_mean(y*tf.log(clip(layer2['output'])))

with tf.name_scope('train'):
    run_train = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

with tf.name_scope('Accuracy'):
    correct_prediction = tf.equal(tf.argmax(layer2['output'], 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

tf.summary.scalar("cost_track", cross_entropy)
tf.summary.scalar("accuracy", accuracy)
tf.summary.histogram("weights_hist",layer1['w'])
tf.summary.histogram("biases_hist",layer2['w'])

summary_op = tf.summary.merge_all()

In [8]:
print_var("trainable_variables")

layer1/weight:0 : [784, 300]
layer1/bias:0 : [300]
layer2/weight:0 : [300, 10]
layer2/bias:0 : [10]


In [9]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
writer = tf.summary.FileWriter(logs_path, sess.graph)

for epoch_ in range(epoch):
    index = np.arange(len(train['input']))
    np.random.shuffle(index)
    trX = train['input'][index]
    trY = train['output'][index]
    
    for nbatch_ in range(nbatch):
        train_feed_dict = {x : trX[batch_size*nbatch_:batch_size*(nbatch_+1)], 
                           y : trY[batch_size*nbatch_:batch_size*(nbatch_+1)]}
        _, summary = sess.run([run_train, summary_op], feed_dict=train_feed_dict)
        writer.add_summary(summary, epoch_ * nbatch+ nbatch_)

In [10]:
tf.get_default_graph()

In [11]:
sess.graph